In [ ]:
#Imports for M-LOOP
# import mloop.interfaces as mli
# import mloop.controllers as mlc
# import mloop.visualizations as mlv

#Other imports
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm.notebook import trange
# import time

from functions import*
from Mloop_and_simcim import*

datatype = torch.float32
device = 'cuda'

In [ ]:
# define number of cities
N_cities = 40
# random generation of city coordinates and their lengths
cities,lengths = generate_cities(N_cities)


B = 0.1
A = 1.5*B*lengths.max()
J,h = get_Jh(lengths, A, B)
J = J.to(device)
h = h.to(device)

# ['gaussian_process', 'neural_net', 'differential_evolution', 'nelder_mead', 'random']
controller_type = 'neural_net'
simcim = Simcim(J, h, device, datatype)
simcim.params_disc['attempt_num'] = 100
simcim.params_cont['O'] = 0.1
simcim.params_cont['S'] = 0.2
simcim.params_cont['D'] = -0.
simcim.params_cont['sigma'] = 1.
simcim.params_cont['dt'] = 0.1
params_opt = main(controller_type,simcim)
c_current, c_evol = simcim.evolve(params_opt)

for i in range(c_evol.shape[0]):
    plt.plot(c_evol[i].cpu().numpy())
plt.show()
s_cur = torch.sign(c_current)
E = energy(J,h,s_cur)
plt.hist(E.cpu().numpy())
plt.show()

s_min = s_cur[:,torch.argmin(E)]
order = get_order_simcim(s_min,N_cities)

fig = plot_cities(cities,lengths,order.cpu().numpy())

In [ ]:
simcim.params_disc['attempt_num'] = 1000
c_current, c_evol = simcim.evolve(params_opt)

s_min = s_cur[:,torch.argmin(E)]
order = get_order_simcim(s_min,N_cities)

fig = plot_cities(cities,lengths,order.cpu().numpy())